In [ ]:
import cv2
import numpy as np
from collections import deque
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f" GPU detected and enabled: {gpus}")
    except RuntimeError as e:
        print(f"GPU setup error: {e}")
else:
    print(" No GPU detected. Running on CPU.")
    print("  For GPU support, install: pip install tensorflow[and-cuda]")

from deepface import DeepFace

EMOTIONS = ["angry", "disgust", "fear", "happy", "sad", "surprise", "neutral"]

EMOTION_COLORS = {
    "angry": (0, 0, 255),
    "disgust": (0, 128, 0),
    "fear": (128, 0, 128),
    "happy": (0, 255, 255),
    "sad": (255, 0, 0),
    "surprise": (0, 165, 255),
    "neutral": (200, 200, 200)
}

emotion_history = deque(maxlen=5)

def smooth_emotions(current_emotions):
    emotion_history.append(current_emotions)

    if len(emotion_history) < 2:
        return current_emotions

    smoothed = {}
    for emotion in EMOTIONS:
        values = [h.get(emotion, 0) for h in emotion_history]
        smoothed[emotion] = sum(values) / len(values)

    return smoothed

def draw_rounded_rect(img, pt1, pt2, color, thickness, radius=15):
    x1, y1 = pt1
    x2, y2 = pt2

    cv2.line(img, (x1 + radius, y1), (x2 - radius, y1), color, thickness)
    cv2.line(img, (x1 + radius, y2), (x2 - radius, y2), color, thickness)
    cv2.line(img, (x1, y1 + radius), (x1, y2 - radius), color, thickness)
    cv2.line(img, (x2, y1 + radius), (x2, y2 - radius), color, thickness)

    cv2.ellipse(img, (x1 + radius, y1 + radius), (radius, radius), 180, 0, 90, color, thickness)
    cv2.ellipse(img, (x2 - radius, y1 + radius), (radius, radius), 270, 0, 90, color, thickness)
    cv2.ellipse(img, (x1 + radius, y2 - radius), (radius, radius), 90, 0, 90, color, thickness)
    cv2.ellipse(img, (x2 - radius, y2 - radius), (radius, radius), 0, 0, 90, color, thickness)

def draw_emotion_panel(frame, emotions_dict, x, y):
    panel_width = 200
    panel_height = 200

    overlay = frame.copy()
    cv2.rectangle(overlay, (x, y), (x + panel_width, y + panel_height), (30, 30, 30), -1)
    cv2.addWeighted(overlay, 0.7, frame, 0.3, 0, frame)

    cv2.putText(frame, "EMOTIONS", (x + 10, y + 25),cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    sorted_emotions = sorted(emotions_dict.items(), key=lambda x: -x[1])

    bar_height = 18
    bar_max_width = 120
    start_y = y + 45

    for i, (emotion, score) in enumerate(sorted_emotions):
        bar_y = start_y + (i * (bar_height + 6))

        cv2.putText(frame, emotion.capitalize()[:3], (x + 10, bar_y + 14),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.45, (180, 180, 180), 1)

        cv2.rectangle(frame, (x + 45, bar_y), (x + 45 + bar_max_width, bar_y + bar_height),
                      (60, 60, 60), -1)

        filled_width = int(bar_max_width * score)
        if filled_width > 0:
            color = EMOTION_COLORS.get(emotion, (0, 255, 0))
            cv2.rectangle(frame, (x + 45, bar_y), (x + 45 + filled_width, bar_y + bar_height),
                          color, -1)

        cv2.putText(frame, f"{score:.0%}", (x + 170, bar_y + 14),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)

def main():
    print("=" * 60)
    print("   FACE & EMOTION DETECTION - ACCESSIBILITY MODE")
    print("   Using DeepFace with RetinaFace (High Accuracy)")
    print("=" * 60)
    print("Detectable emotions:", ", ".join(EMOTIONS))
    print("Press 'Q' to quit")
    print("=" * 60)

    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    cap.set(cv2.CAP_PROP_FPS, 30)
    cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)

    frame_count = 0
    analyze_every = 3

    last_result = None
    last_face_region = None

    print("\nInitializing... First detection may take a few seconds.\n")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        frame_count += 1

        h, w = frame.shape[:2]

        if frame_count % analyze_every == 0:
            try:
                results = DeepFace.analyze(
                    frame,
                    actions=['emotion'],
                    detector_backend='retinaface',
                    enforce_detection=False,
                    silent=True
                )

                if results and isinstance(results, list):
                    last_result = results[0]
                    last_face_region = last_result.get('region', None)
                elif results and isinstance(results, dict):
                    last_result = results
                    last_face_region = last_result.get('region', None)

            except Exception as e:
                try:
                    results = DeepFace.analyze(
                        frame,
                        actions=['emotion'],
                        detector_backend='opencv',
                        enforce_detection=False,
                        silent=True
                    )
                    if results:
                        last_result = results[0] if isinstance(results, list) else results
                        last_face_region = last_result.get('region', None)
                except:
                    pass

        if last_result and last_face_region:
            x = last_face_region.get('x', 0)
            y_pos = last_face_region.get('y', 0)
            face_w = last_face_region.get('w', 100)
            face_h = last_face_region.get('h', 100)

            emotion_scores = last_result.get('emotion', {})

            normalized_scores = {}
            for emotion in EMOTIONS:
                score = emotion_scores.get(emotion, 0)
                normalized_scores[emotion] = score / 100.0 if score > 1 else score

            smoothed_scores = smooth_emotions(normalized_scores)

            dominant_emotion = max(smoothed_scores.items(), key=lambda x: x[1])
            emotion_name, emotion_score = dominant_emotion

            box_color = EMOTION_COLORS.get(emotion_name, (0, 255, 0))

            padding = 20
            x1 = max(0, x - padding)
            y1 = max(0, y_pos - padding)
            x2 = min(w, x + face_w + padding)
            y2 = min(h, y_pos + face_h + padding)

            cv2.rectangle(frame, (x1, y1), (x2, y2), box_color, 3)

            corner_len = 30
            thickness = 5
            cv2.line(frame, (x1, y1), (x1 + corner_len, y1), box_color, thickness)
            cv2.line(frame, (x1, y1), (x1, y1 + corner_len), box_color, thickness)
            cv2.line(frame, (x2, y1), (x2 - corner_len, y1), box_color, thickness)
            cv2.line(frame, (x2, y1), (x2, y1 + corner_len), box_color, thickness)
            cv2.line(frame, (x1, y2), (x1 + corner_len, y2), box_color, thickness)
            cv2.line(frame, (x1, y2), (x1, y2 - corner_len), box_color, thickness)
            cv2.line(frame, (x2, y2), (x2 - corner_len, y2), box_color, thickness)
            cv2.line(frame, (x2, y2), (x2, y2 - corner_len), box_color, thickness)

            label = f"{emotion_name.upper()} {emotion_score:.0%}"
            label_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 1.2, 3)[0]

            label_x = x1
            label_y = y1 - 15

            cv2.rectangle(frame,
                          (label_x - 5, label_y - label_size[1] - 10),
                          (label_x + label_size[0] + 10, label_y + 5),
                          box_color, -1)

            cv2.putText(frame, label, (label_x, label_y),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 0), 3)

            panel_x = min(x2 + 30, w - 220)
            panel_y = max(y1, 10)
            draw_emotion_panel(frame, smoothed_scores, panel_x, panel_y)

        else:
            warning_text = "Looking for face... Please face the camera"
            text_size = cv2.getTextSize(warning_text, cv2.FONT_HERSHEY_SIMPLEX, 0.8, 2)[0]
            text_x = (w - text_size[0]) // 2

            cv2.rectangle(frame, (text_x - 10, 30), (text_x + text_size[0] + 10, 70), (0, 0, 150), -1)
            cv2.putText(frame, warning_text, (text_x, 55),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

        info_text = "Press 'Q' to quit | DeepFace + RetinaFace | High Accuracy Mode"
        cv2.putText(frame, info_text, (10, h - 15),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (150, 150, 150), 1)

        cv2.imshow("Face & Emotion Detection - Accessibility", frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q') or key == ord('Q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    print("\n Application closed.")

if __name__ == "__main__":
    main()


 No GPU detected. Running on CPU.
  For GPU support, install: pip install tensorflow[and-cuda]
26-01-09 09:16:15 - Directory /home/rama/.deepface has been created
26-01-09 09:16:15 - Directory /home/rama/.deepface/weights has been created
   FACE & EMOTION DETECTION - ACCESSIBILITY MODE
   Using DeepFace with RetinaFace (High Accuracy)
Detectable emotions: angry, disgust, fear, happy, sad, surprise, neutral
Press 'Q' to quit

Initializing... First detection may take a few seconds.


 Application closed.


[ WARN:0@5844.725] global cap_v4l.cpp:914 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@5844.726] global obsensor_uvc_stream_channel.cpp:163 getStreamChannelGroup Camera index out of range
